# Transformers for recommendations with Replay

В качестве референса был использован код из данного репозитория:https://github.com/RukhovichIV/RePlay-paper-listings/blob/main/compare/Netflix/run_replay.py#L90

Состав команды: ...

## Подготовка окружения 

Ставим окружение если не установлено 

In [1]:
%pip install -U pip

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 26.8 MB/s  0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.12 requires torch<2.1,>=1.7, but you have torch 2.4.1+cu121 which is incompatible.

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
%pip install lightning==2.3.3
%pip install pandas==1.5.3
%pip install replay-rec==0.20.0
%pip install torch==2.4.1
%pip install transformers==4.57.1

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
# cпециально убиваем kernel и перезапускаем чтобы подтянулось скачанное выше окружение 
import os, signal 
os.kill(os.getpid(), signal.SIGKILL) 

[0;31mKernelNotResponding[0m: The kernel died unexpectedly and has been restarted. If the restart fails, restart the kernel from the Kernel menu.

In [202]:
# импорт всего необходимого

import os
import random
import warnings

import numpy as np
import pandas as pd
import torch
import lightning as L
from lightning.pytorch.loggers import CSVLogger
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping

from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.lr_scheduler import ReduceLROnPlateau


from replay.metrics import Recall
from replay.splitters import LastNSplitter
from replay.data import (
    FeatureHint,
    FeatureInfo,
    FeatureSchema,
    FeatureSource,
    FeatureType,
    Dataset,
)
from replay.models.nn.optimizer_utils import FatOptimizerFactory
from replay.models.nn.sequential.callbacks import (
    ValidationMetricsCallback,
    PandasPredictionCallback,
)
from replay.models.nn.sequential.postprocessors import RemoveSeenItems
from replay.data.nn import (
    SequenceTokenizer,
    SequentialDataset,
    TensorFeatureSource,
    TensorSchema,
    TensorFeatureInfo,
)
from replay.models.nn.sequential import Bert4Rec, SasRec
from replay.models.nn.sequential.sasrec import (SasRecPredictionDataset,
                                                SasRecTrainingDataset,
                                                SasRecValidationDataset)
from replay.models.nn.sequential.bert4rec import (
    Bert4RecModel,
    Bert4RecTrainingDataset,
    Bert4RecValidationDataset,
    Bert4RecPredictionDataset,
    Bert4RecPredictionBatch,
)

warnings.filterwarnings("ignore")

/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:232: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  language = self._parent_header['metadata'].get('language')
/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:237: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  livy = self._parent_header['metadata'].get('livy')
/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:280: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  header = self._parent_header['header']


In [203]:
def seed_everything(seed: int = 42) -> None:
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'
    os.environ['OMP_NUM_THREADS'] = '1'
    os.environ['MKL_NUM_THREADS'] = '1'
    os.environ['OPENBLAS_NUM_THREADS'] = '1'
    os.environ['NUMEXPR_NUM_THREADS'] = '1'

    random.seed(seed)
    np.random.seed(seed)

    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.use_deterministic_algorithms(True, warn_only=True)
    torch.backends.cuda.matmul.allow_tf32 = False
    torch.backends.cudnn.allow_tf32 = False
    torch.use_deterministic_algorithms(False)
    torch.backends.cudnn.benchmark = True
    torch.set_float32_matmul_precision('high')

    L.seed_everything(seed, workers=True)

seed_everything(42)

Seed set to 42


In [204]:
interactions = pd.read_csv('hse-rec-sys-challenge-2025/events.csv')
users = pd.read_csv('hse-rec-sys-challenge-2025/user_features.csv')
items = pd.read_csv('hse-rec-sys-challenge-2025/item_features.csv')

## Feature Engeneering and Dataset Creation

In [205]:
splitter = LastNSplitter(
    divide_column='user_id',
    query_column='user_id',
    item_column='item_id',
    timestamp_column='timestamp',
    strategy='interactions',
    N=1,
)

raw_test_events, raw_test_gt = splitter.split(interactions)
raw_train_events, raw_validation_gt = splitter.split(raw_test_events)

In [206]:
def infer_genre_cols(items_df):
    return [c for c in items_df.columns if c.startswith('genre_')]

def build_user_item_features_discrete(
    base_events: pd.DataFrame,
    users_df: pd.DataFrame,
    items_df: pd.DataFrame,
    ts_col: str = 'timestamp',
    user_col: str = 'user_id',
    item_col: str = 'item_id',
    rating_col: str = 'rating',
    recent_last_n: int | None = None,
    recent_frac: float = 0.1,
    bayes_alpha: float = 5.0 
):
    df = base_events[[user_col, item_col, rating_col, ts_col]].copy()

    now_ts = int(df[ts_col].max()) if len(df) else 0
    min_ts = int(df[ts_col].min()) if len(df) else 0
    total_span = max(1, now_ts - min_ts + 1)

    if recent_last_n is None:
        recent_last_n = max(1, int(total_span * recent_frac))
    recent_cut = now_ts - recent_last_n + 1
    recent_mask = df[ts_col] >= recent_cut

    global_mean = df[rating_col].mean() if len(df) else 0.0

    u_agg = df.groupby(user_col).agg(
        user_interactions=(item_col, 'size'),
        user_unique_items=(item_col, 'nunique'),
        user_avg_rating=(rating_col, 'mean'),
        user_std_rating=(rating_col, 'std'),
        user_last_ts=(ts_col, 'max'),
    ).reset_index()
    u_agg['user_std_rating'] = u_agg['user_std_rating'].fillna(0.0)
    u_agg['user_recency_pos'] = now_ts - u_agg['user_last_ts']

    u_recent = (
        df.loc[recent_mask]
          .groupby(user_col)
          .agg(user_recent_interactions=(item_col, 'size'))
          .reset_index()
    )
    u_agg = u_agg.merge(u_recent, on=user_col, how='left')
    u_agg['user_recent_interactions'] = u_agg['user_recent_interactions'].fillna(0).astype(int)

    genre_cols = infer_genre_cols(items_df)
    if genre_cols:
        tmp = df.merge(items_df[[item_col] + genre_cols], on=item_col, how='left')
        u_genres = tmp.groupby(user_col)[genre_cols].mean().add_prefix('user_pref_').reset_index()
        u_agg = u_agg.merge(u_genres, on=user_col, how='left')

    users_aug = users_df.copy()
    if 'gender' in users_aug.columns:
        g = pd.get_dummies(users_aug['gender'].fillna('UNK'), prefix='user_gender')
        users_aug = pd.concat([users_aug.drop(columns=['gender']), g], axis=1)
    if 'age' in users_aug.columns:
        bins = [-1, 17, 24, 34, 44, 54, 64, 150]
        labels = ['age_0_17','age_18_24','age_25_34','age_35_44','age_45_54','age_55_64','age_65+']
        users_aug['age_bin'] = pd.cut(users_aug['age'].fillna(-1), bins=bins, labels=labels)
        users_aug = pd.concat([users_aug, pd.get_dummies(users_aug['age_bin'], prefix='', prefix_sep='')], axis=1).drop(columns=["age_bin"])

    user_features = users_aug.merge(u_agg, on=user_col, how='left')
    num_cols_u = user_features.select_dtypes(include=[np.number]).columns.tolist()
    user_features[num_cols_u] = user_features[num_cols_u].fillna(0)

    i_agg = df.groupby(item_col).agg(
        item_interactions=(user_col, 'size'),
        item_unique_users=(user_col, 'nunique'),
        item_avg_rating=(rating_col, 'mean'),
        item_std_rating=(rating_col, 'std'),
        item_last_ts=(ts_col, 'max'),
    ).reset_index()
    i_agg['item_std_rating'] = i_agg['item_std_rating'].fillna(0.0)
    i_agg['item_recency_pos'] = now_ts - i_agg['item_last_ts']

    i_recent = (
        df.loc[recent_mask]
          .groupby(item_col)
          .agg(item_recent_interactions=(user_col, 'size'))
          .reset_index()
    )
    i_agg = i_agg.merge(i_recent, on=item_col, how='left')
    i_agg['item_recent_interactions'] = i_agg['item_recent_interactions'].fillna(0).astype(int)

    i_agg['item_popularity'] = i_agg['item_interactions']
    if len(i_agg):
        i_agg['item_popularity_norm'] = (i_agg['item_popularity'] - i_agg['item_popularity'].mean()) / (i_agg['item_popularity'].std() + 1e-9)
        i_agg['item_popularity_rank'] = i_agg['item_popularity'].rank(ascending=False, method='dense').astype(int)
    else:
        i_agg['item_popularity_norm'] = 0.0
        i_agg['item_popularity_rank'] = 0

    i_agg['item_bayesian_rating'] = (
        (i_agg['item_avg_rating'] * i_agg['item_interactions'] + global_mean * bayes_alpha)
        / (i_agg['item_interactions'] + bayes_alpha)
    )

    item_features = items_df.merge(i_agg, on=item_col, how='left')
    num_cols_i = item_features.select_dtypes(include=[np.number]).columns.tolist()
    item_features[num_cols_i] = item_features[num_cols_i].fillna(0)

    return user_features, item_features


# Фичи для train/val — только на raw_train_events
train_user_features, train_item_features = build_user_item_features_discrete(
    base_events=raw_train_events,
    users_df=users,
    items_df=items,
    ts_col='timestamp',
    user_col='user_id',
    item_col='item_id',
    rating_col='rating',
    recent_last_n= max(1, int((raw_train_events['timestamp'].max() - raw_train_events['timestamp'].min() + 1) * 0.1))
)

# Для test — на raw_test_events (т.е. train + val-события)
test_user_features, test_item_features = build_user_item_features_discrete(
    base_events=raw_test_events,
    users_df=users,
    items_df=items,
    ts_col='timestamp',
    user_col='user_id',
    item_col='item_id',
    rating_col='rating',
    recent_last_n= max(1, int((raw_test_events['timestamp'].max() - raw_test_events['timestamp'].min() + 1) * 0.1))
)

print('Train user_features:', train_user_features.shape, 'item_features:', train_item_features.shape)
print('Test  user_features:', test_user_features.shape,  'item_features:', test_item_features.shape)


Train user_features: (6040, 36) item_features: (3706, 30)
Test  user_features: (6040, 36) item_features: (3706, 30)


In [207]:
def prepare_feature_schema(is_ground_truth: bool) -> FeatureSchema:
    base_features = FeatureSchema(
        [
            FeatureInfo(
                column="user_id",
                feature_hint=FeatureHint.QUERY_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
            FeatureInfo(
                column="item_id",
                feature_hint=FeatureHint.ITEM_ID,
                feature_type=FeatureType.CATEGORICAL,
            ),
        ]
    )
    if is_ground_truth:
        return base_features

    all_features = base_features + FeatureSchema(
        [
            FeatureInfo(
                column="timestamp",
                feature_type=FeatureType.NUMERICAL,
                feature_hint=FeatureHint.TIMESTAMP,
            ),
        ]
    )
    return all_features

In [208]:
train_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_train_events,
    check_consistency=True,
    categorical_encoded=False,
    item_features=train_item_features,
    query_features=train_user_features,
)
validation_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_train_events,
    check_consistency=True,
    categorical_encoded=False,
    item_features=train_item_features,
    query_features=train_user_features,
)
validation_gt = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=True),
    interactions=raw_validation_gt,
    check_consistency=True,
    categorical_encoded=False,
    item_features=train_item_features,
    query_features=train_user_features,
)
test_dataset = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=False),
    interactions=raw_test_events,
    check_consistency=True,
    categorical_encoded=False,
    item_features=test_item_features,
    query_features=test_user_features,
)
test_gt = Dataset(
    feature_schema=prepare_feature_schema(is_ground_truth=True),
    interactions=raw_test_gt,
    check_consistency=True,
    categorical_encoded=False,
    item_features=test_item_features,
    query_features=test_user_features,
)

In [209]:
ITEM_FEATURE_NAME = 'item_id_seq'
HIDDEN_DIM = 328

tensor_schema = TensorSchema(
    TensorFeatureInfo(
        name=ITEM_FEATURE_NAME,
        feature_type=FeatureType.CATEGORICAL,
        is_seq=True,
        feature_hint=FeatureHint.ITEM_ID,
        feature_sources=[TensorFeatureSource(FeatureSource.INTERACTIONS, train_dataset.feature_schema.item_id_column)],
        embedding_dim=HIDDEN_DIM
    )
)

In [210]:
tokenizer = SequenceTokenizer(tensor_schema, allow_collect_to_master=True)
tokenizer.fit(train_dataset)

sequential_train_dataset = tokenizer.transform(train_dataset)
sequential_test_dataset = tokenizer.transform(test_dataset)

sequential_val_gt = tokenizer.transform(validation_gt, [tensor_schema.item_id_feature_name])
sequential_val_dataset, sequential_val_gt = SequentialDataset.keep_common_query_ids(
    sequential_train_dataset, sequential_val_gt
)

sequential_test_gt = tokenizer.transform(test_gt, [tensor_schema.item_id_feature_name])
sequential_test_dataset, sequential_test_gt = SequentialDataset.keep_common_query_ids(
    sequential_test_dataset, sequential_test_gt
)

## Подготовка модели 

In [211]:
MAX_SEQ_LEN = 128
BATCH_SIZE = 512
NUM_WORKERS = 4
NUM_EPOCHS = 1000
BLOCK_COUNT = 8
HEAD_COUNT = 2

monitor = "recall@10"

In [212]:
class ReduceOnPlateauFactory:
    def __init__(
        self,
        mode: str = "min",
        factor: float = 0.9,
        patience: int = 2,
        threshold: float = 1e-3,
        threshold_mode: str = "rel",
        cooldown: int = 0,
        min_lr: float = 1e-6,
        eps: float = 1e-8,
        verbose: bool = False,
        monitor: str = "val_loss",
        interval: str = "epoch",
        frequency: int = 1,
        strict: bool = True,
    ):
        self.scheduler_kwargs = dict(
            mode=mode, factor=factor, patience=patience, threshold=threshold,
            threshold_mode=threshold_mode, cooldown=cooldown, min_lr=min_lr,
            eps=eps, verbose=verbose,
        )
        self.monitor = monitor
        self.interval = interval
        self.frequency = frequency
        self.strict = strict

    def create(self, optimizer):
        sch = ReduceLROnPlateau(optimizer, **self.scheduler_kwargs)
        return {
            "scheduler": sch,
            "monitor": self.monitor,
            "interval": self.interval,
            "frequency": self.frequency,
            "strict": self.strict,
        }


In [213]:
class CosineFactory:
    def __init__(self, T_max: int, eta_min=5e-4):
        self.T_max = T_max
        self.eta_min = eta_min

    def create(self, optimizer):
        return CosineAnnealingLR(optimizer, T_max=self.T_max, eta_min=self.eta_min)

In [214]:
model = Bert4Rec(
                 tensor_schema,
                 block_count=BLOCK_COUNT,
                 head_count=HEAD_COUNT,
                 max_seq_len=MAX_SEQ_LEN,
                 hidden_size=HIDDEN_DIM,
                 dropout_rate=0.2,
                 lr_scheduler_factory=ReduceOnPlateauFactory(factor=0.98, patience=5, min_lr=1e-6, monitor=monitor),
                 optimizer_factory=FatOptimizerFactory(learning_rate=0.001)
)

In [215]:
csv_logger = CSVLogger(save_dir=".logs/train", name="My_model")

In [216]:


checkpoint_callback = ModelCheckpoint(dirpath=".checkpoints",
                                      save_top_k=1,
                                      verbose=True,
                                      monitor=monitor,
                                      mode="max")

validation_metrics_callback = ValidationMetricsCallback(metrics=["map", "ndcg", "recall"],
                                                        ks=[10],
                                                        item_count=train_dataset.item_count,
                                                        postprocessors=[RemoveSeenItems(sequential_val_dataset)])

In [217]:
early_stop_callback = EarlyStopping(
    monitor=monitor,
    min_delta=0.0005,
    patience=70,
    verbose=True,
    mode="max"
)

In [218]:
trainer = L.Trainer(max_epochs=NUM_EPOCHS,
                    callbacks=[checkpoint_callback, 
                               validation_metrics_callback,
                               early_stop_callback
                              ],
                    logger=csv_logger,
                    precision='16-mixed',
                    gradient_clip_val=1,
                   accumulate_grad_batches=1)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [219]:
train_dataloader = DataLoader(
    dataset=Bert4RecTrainingDataset(
        sequential_train_dataset,
        max_sequence_length=MAX_SEQ_LEN,
        mask_prob=0.15),
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    persistent_workers=True,
    multiprocessing_context="spawn"
)

validation_dataloader = DataLoader(
    dataset=Bert4RecValidationDataset(
        sequential_val_dataset,
        sequential_val_gt,
        sequential_train_dataset,
        max_sequence_length=MAX_SEQ_LEN),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True,
    persistent_workers=True,
    multiprocessing_context="spawn"
)

/tmp/ipykernel_8737/1941269140.py:2: DeprecationWarning: Bert4RecTrainingDataset.__init__ `padding_value` parameter will be removed in future versions. Instead, you should specify `padding_value` for each column in TensorSchema
  dataset=Bert4RecTrainingDataset(
/home/jupyter/.local/lib/python3.10/site-packages/replay/models/nn/sequential/bert4rec/dataset.py:141: DeprecationWarning: TorchSequentialDataset.__init__ `padding_value` parameter will be removed in future versions. Instead, you should specify `padding_value` for each column in TensorSchema
  self._inner = TorchSequentialDataset(
/tmp/ipykernel_8737/1941269140.py:15: DeprecationWarning: Bert4RecValidationDataset.__init__ `padding_value` parameter will be removed in future versions. Instead, you should specify `padding_value` for each column in TensorSchema
  dataset=Bert4RecValidationDataset(
/home/jupyter/.local/lib/python3.10/site-packages/replay/models/nn/sequential/bert4rec/dataset.py:267: DeprecationWarning: TorchSequenti

In [220]:
model = torch.compile(model)

/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:232: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  language = self._parent_header['metadata'].get('language')
/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:237: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  livy = self._parent_header['metadata'].get('livy')
/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:280: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  header = self._parent_header['header']


## Обучение модели

In [221]:
trainer.fit(
    model,
    train_dataloaders=train_dataloader,
    val_dataloaders=validation_dataloader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type             | Params | Mode 
----------------------------------------------------
0 | _model | Bert4RecModel    | 12.8 M | train
1 | _loss  | CrossEntropyLoss | 0      | train
----------------------------------------------------
12.8 M    Trainable params
0         Non-trainable params
12.8 M    Total params
51.357    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/home/jupyter/.local/lib/python3.10/site-packages/replay/models/optimization/optuna_mixin.py:240: FeatureUnavailableWarning: Optimization feature not enabled - `optuna` package not found. Ensure you have the package installed if you want to use the `optimize()` method in your recommenders.
  warnings.warn(feature_warning)
/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:110: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use 

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

W1022 20:23:43.865000 140127755515008 torch/fx/experimental/symbolic_shapes.py:4449] [7/16] r0 is not in var_ranges, defaulting to unknown range.
W1022 20:23:43.866000 140127755515008 torch/fx/experimental/symbolic_shapes.py:4449] [7/16] r2 is not in var_ranges, defaulting to unknown range.
W1022 20:23:43.880000 140127755515008 torch/fx/experimental/symbolic_shapes.py:4449] [7/16] r1 is not in var_ranges, defaulting to unknown range.
W1022 20:23:43.881000 140127755515008 torch/fx/experimental/symbolic_shapes.py:4449] [7/16] r0 is not in var_ranges, defaulting to unknown range.
W1022 20:23:43.881000 140127755515008 torch/fx/experimental/symbolic_shapes.py:4449] [7/16] r2 is not in var_ranges, defaulting to unknown range.
W1022 20:23:43.890000 140127755515008 torch/fx/experimental/symbolic_shapes.py:4449] [7/16] r0 is not in var_ranges, defaulting to unknown range.
W1022 20:23:43.890000 140127755515008 torch/fx/experimental/symbolic_shapes.py:4449] [7/16] r2 is not in var_ranges, default

...

Epoch 540: 100%|██████████| 12/12 [00:04<00:00,  2.72it/s, v_num=62, train_loss_step=4.940, train_loss_epoch=4.970]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████| 12/12 [00:01<00:00,  8.96it/s]k             10
map     0.109677
ndcg    0.152581
recall  0.294040


Epoch 540: 100%|██████████| 12/12 [00:05<00:00,  2.02it/s, v_num=62, train_loss_step=4.940, train_loss_epoch=4.960]

Epoch 540, global step 6492: 'recall@10' was not in top 1


Epoch 541: 100%|██████████| 12/12 [00:04<00:00,  2.70it/s, v_num=62, train_loss_step=4.910, train_loss_epoch=4.960]
Validation: |          | 0/? [00:00<?, ?it/s]
Validation DataLoader 0: 100%|██████████| 12/12 [00:01<00:00,  8.97it/s]k             10
map     0.109559
ndcg    0.152263
recall  0.293046


Epoch 541: 100%|██████████| 12/12 [00:05<00:00,  2.02it/s, v_num=62, train_loss_step=4.910, train_loss_epoch=4.970]

Monitored metric recall@10 did not improve in the last 70 records. Best score: 0.301. Signaling Trainer to stop.
Epoch 541, global step 6504: 'recall@10' was not in top 1


Epoch 541: 100%|██████████| 12/12 [00:05<00:00,  2.01it/s, v_num=62, train_loss_step=4.910, train_loss_epoch=4.970]


In [222]:
best_model = Bert4Rec \
            .load_from_checkpoint(checkpoint_callback.best_model_path)

/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:232: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  language = self._parent_header['metadata'].get('language')
/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:237: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  livy = self._parent_header['metadata'].get('livy')
/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:280: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  header = self._parent_header['header']


In [223]:
torch.save(best_model.state_dict(), "./my_model.pth")

## Инференс модели

In [224]:
prediction_dataloader = DataLoader(
    dataset=Bert4RecPredictionDataset(
        sequential_test_dataset,
        max_sequence_length=MAX_SEQ_LEN),
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=True)

csv_logger = CSVLogger(save_dir=".logs/test", name="RecommenderBERTModelv7")

/tmp/ipykernel_8737/2860318794.py:2: DeprecationWarning: Bert4RecPredictionDataset.__init__ `padding_value` parameter will be removed in future versions. Instead, you should specify `padding_value` for each column in TensorSchema
  dataset=Bert4RecPredictionDataset(
/home/jupyter/.local/lib/python3.10/site-packages/replay/models/nn/sequential/bert4rec/dataset.py:201: DeprecationWarning: TorchSequentialDataset.__init__ `padding_value` parameter will be removed in future versions. Instead, you should specify `padding_value` for each column in TensorSchema
  self._inner = TorchSequentialDataset(


In [225]:
TOPK = [10]

pandas_prediction_callback = PandasPredictionCallback(
    top_k=max(TOPK),
    query_column="user_id",
    item_column="item_id",
    rating_column="score",
    postprocessors=[RemoveSeenItems(sequential_test_dataset)],
)

/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:232: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  language = self._parent_header['metadata'].get('language')
/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:237: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  livy = self._parent_header['metadata'].get('livy')
/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:280: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  header = self._parent_header['header']


In [226]:
predictor = L.Trainer(
    callbacks=[
        pandas_prediction_callback,
    ],
    logger=csv_logger,
    inference_mode=True
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [227]:
predictor.predict(best_model, dataloaders=prediction_dataloader, return_predictions=False)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 12/12 [00:01<00:00,  6.08it/s]


In [228]:
pandas_res = pandas_prediction_callback.get_result()

/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:232: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  language = self._parent_header['metadata'].get('language')
/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:237: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  livy = self._parent_header['metadata'].get('livy')
/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:280: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  header = self._parent_header['header']


In [229]:
recommendations = tokenizer.query_and_item_id_encoder.inverse_transform(pandas_res)

/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:232: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  language = self._parent_header['metadata'].get('language')
/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:237: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  livy = self._parent_header['metadata'].get('livy')
/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:280: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  header = self._parent_header['header']


In [230]:
recommendations = recommendations.sort_values(by=['user_id', 'score'], ascending=[True, False])

recommendations_grouped = recommendations.groupby('user_id').apply(
lambda x: ' '.join(x['item_id'].head(10).astype(str)))

final_df = recommendations_grouped.reset_index(name='item_id')
final_df

,user_id,item_id
0,0,1461 3271 1058 2454 69 450 3421 2690 3222 84
1,1,232 1822 1246 2084 350 1813 3101 3205 2206 1686
2,2,2428 382 2774 221 2311 2354 234 724 1560 1781
3,3,1560 3013 1316 3272 989 467 2564 1223 3035 487
4,4,1086 1337 702 3035 1545 2185 2814 1868 802 2653
...,...,...
6035,6035,2256 2054 476 2646 2664 2606 2148 82 3382 1403
6036,6036,3142 1560 3153 2603 772 2402 1354 3473 3692 2688
6037,6037,1296 1975 2256 2776 2056 2502 2054 1968 1797 2732
6038,6038,3278 2412 3061 1680 976 1357 185 2354 705 1794


In [231]:
final_df.to_csv('submit.csv', index=False, header=True)

/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:232: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  language = self._parent_header['metadata'].get('language')
/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:237: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  livy = self._parent_header['metadata'].get('livy')
/kernel/lib/python3.10/site-packages/ml_kernel/kernel.py:280: DeprecationWarning: Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  header = self._parent_header['header']
